# Environmental Impact Evaluation of Concrete Frame Bridges

This notebook takes the parameter vectors x as input and calculates first the volumes and the environmental impacts of all the structures.

In [105]:
import pickle
import pandas as pd
import os
import matplotlib
import matplotlib.pyplot as plt
import math as m

In [106]:
idx_s=200


# Import CFM_xy data
xy_file_name='CFBData\\{}_Batch\\{}_CFB_xy.csv'.format(idx_s,idx_s)

df_xy=pd.read_csv(xy_file_name)
df_xy

,Unnamed: 0,ID,L,b1,t_p,t_w,h_w,d1_plate,d4_plate,s_plate,...,y_s,z_s,Location_s,GP_count_s,eta_min_c,x_c,y_c,z_c,Location_c,GP_count_c
0,0,0,3264.911067,16765.132982,531.656436,621.775968,2604.161687,24.228625,13.427572,206.036846,...,NaN,NaN,NaN,NaN,0.000000,NaN,NaN,NaN,NaN,NaN
1,1,1,5361.115418,17451.415474,744.304389,275.279703,3820.154356,17.838816,12.481430,218.545430,...,4987.521973,-2517.141846,eta_1_x,0.0,5.430790,173.859085,98.314178,-4067.336670,eta_3_bot,0.0
2,2,2,7197.928187,12610.611102,792.070735,496.244565,4273.476243,22.587214,26.028824,128.850732,...,6500.823242,-1904.297729,eta_4_x,0.0,18.270001,293.142883,6500.823242,-1090.091309,eta_3_bot,0.0
3,3,3,4946.728918,16364.414176,373.131286,477.965439,3363.099660,24.607351,29.281128,135.563137,...,2073.840576,-326.489868,eta_2_y,0.0,13.517034,3468.395752,2073.840576,-326.489868,eta_3_top,0.0
4,4,4,4940.337104,19145.252592,464.963852,412.443855,2135.646626,15.313616,16.445258,79.214303,...,1656.870605,-425.109802,eta_1_x,0.0,17.884413,3381.489746,2101.281494,-425.109802,eta_3_top,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,995,995,3281.367780,15498.452667,577.306324,421.458974,4699.266956,29.736156,27.057742,93.172957,...,163.900711,-2296.886475,eta_1_x,0.0,6.351162,1829.632202,163.900711,-4665.050781,eta_3_bot,0.0
996,996,996,3666.643637,15455.753526,792.299652,471.784421,4931.536191,10.685627,14.709277,194.274833,...,2363.356445,-739.479675,eta_3_y,0.0,11.660351,4830.938477,200.508377,-5082.087891,eta_3_bot,0.0
997,997,997,6184.735919,11369.136905,645.636004,245.825059,3643.241327,28.115256,14.995675,101.134360,...,76.088707,-4161.842285,eta_4_x,0.0,9.986733,11232.991211,76.088707,-4161.842285,eta_3_bot,0.0
998,998,998,7491.926992,9743.524453,776.691431,387.895813,2073.204456,22.060343,17.066350,144.671295,...,6957.100586,-921.270569,eta_4_x,0.0,11.777114,8872.617188,6957.100586,-1316.269409,eta_3_bot,0.0


In [107]:
df_xy['eta_min_s']

0       0.000000
1       1.784327
2       4.509594
3      11.349899
4      10.560859
         ...    
995     9.453438
996    14.502720
997     8.950159
998     5.102009
999     2.629160
Name: eta_min_s, Length: 1000, dtype: float64

In [108]:
df_xy['eta_min_tot']=df_xy[['eta_min_c','eta_min_s']].min(axis=1)
df_xy['eta_min_tot']

0       0.000000
1       1.784327
2       4.509594
3      11.349899
4      10.560859
         ...    
995     6.351162
996    11.660351
997     8.950159
998     5.102009
999     2.629160
Name: eta_min_tot, Length: 1000, dtype: float64

In [109]:
df_xy['d4_walls']

0      21.603165
1      17.741306
2      11.975247
3      14.034145
4      18.349838
         ...    
995    28.842929
996    29.057507
997    19.990455
998    20.186831
999    14.677114
Name: d4_walls, Length: 1000, dtype: float64

In [110]:
# start_ID=0
# end_ID=19
# ID=1

##### Calculate Volumes of Deck Slab of concrete and steel
#volume of Concrete
vol_plate_conc=df_xy['t_p']/1000 * df_xy['L']/1000 * (df_xy['b1'] + df_xy['b2'])/2/1000 #[m3]

#volume of Reinforcement
As_1=((df_xy['d1_plate']/1000)**2)*m.pi/4
n_1=((df_xy['b1']+df_xy['b2'])/2)/df_xy['s_plate']

As_4=((df_xy['d4_plate']/1000)**2)*m.pi/4
n_4=((df_xy['b1']+df_xy['b2'])/2)/df_xy['s_plate']

As_2=((df_xy['d2_plate']/1000)**2)*m.pi/4
n_2=df_xy['L']/df_xy['s_plate']

As_3=((df_xy['d3_plate']/1000)**2)*m.pi/4
n_3=df_xy['L']/df_xy['s_plate']

vol_plate_steel = (As_1*n_1 + As_4*n_4)*df_xy['L']/1000  +  (As_2*n_2+As_3*n_3) * ((df_xy['b1']+df_xy['b2'])/2)/1000 #[m3]
w_plate_steel = vol_plate_steel* 7850   #[kg]      #Unit weight of steel is taken as 7850 kg/m3



##### Calculate Volumes of both walls of concrete and steel
#volume of Concrete
vol_walls_conc=df_xy['t_w']/1000 * df_xy['h_w']/1000 *  (df_xy['b1']+df_xy['b2'])/1000     #[m3]



#volume of Reinforcement
As_1=((df_xy['d1_walls']/1000)**2)*m.pi/4
n_1=((df_xy['b1']+df_xy['b2'])/2)/df_xy['s_walls']

As_4=((df_xy['d4_walls']/1000)**2)*m.pi/4
n_4=((df_xy['b1']+df_xy['b2'])/2)/df_xy['s_walls']

As_2=((df_xy['d2_walls']/1000)**2)*m.pi/4
n_2=df_xy['L']/df_xy['s_walls']

As_3=((df_xy['d3_walls']/1000)**2)*m.pi/4
n_3=df_xy['L']/df_xy['s_walls']

vol_walls_steel = ((As_1*n_1 + As_4*n_4)*df_xy['h_w']/1000)*2  +  (As_2*n_2 + As_3*n_3) * (df_xy['b1']+df_xy['b2'])/1000 #[m3]
w_walls_steel = vol_walls_steel* 7850   #[kg]      #Unit weight of steel is taken as 7850 kg/m3


matQuantity_dict={'vol_plate_conc':vol_plate_conc, 'vol_plate_steel':vol_plate_steel,'w_plate_steel':w_plate_steel,
                    'vol_walls_conc':vol_walls_conc, 'vol_walls_steel':vol_walls_steel, 'w_walls_steel':w_walls_steel }

df_matQ=pd.DataFrame(matQuantity_dict)

df_xyz = pd.concat([df_xy, df_matQ], axis=1)
df_xyz


,Unnamed: 0,ID,L,b1,t_p,t_w,h_w,d1_plate,d4_plate,s_plate,...,z_c,Location_c,GP_count_c,eta_min_tot,vol_plate_conc,vol_plate_steel,w_plate_steel,vol_walls_conc,vol_walls_steel,w_walls_steel
0,0,0,3264.911067,16765.132982,531.656436,621.775968,2604.161687,24.228625,13.427572,206.036846,...,NaN,NaN,NaN,0.000000,29.101102,0.220196,1728.540762,54.292379,0.547000,4293.951771
1,1,1,5361.115418,17451.415474,744.304389,275.279703,3820.154356,17.838816,12.481430,218.545430,...,-4067.336670,eta_3_bot,0.0,1.784327,69.636413,0.256209,2011.240414,36.704199,0.383421,3009.852814
2,2,2,7197.928187,12610.611102,792.070735,496.244565,4273.476243,22.587214,26.028824,128.850732,...,-1090.091309,eta_3_bot,0.0,4.509594,71.896477,0.816468,6409.277052,53.486378,0.474510,3724.902312
3,3,3,4946.728918,16364.414176,373.131286,477.965439,3363.099660,24.607351,29.281128,135.563137,...,-326.489868,eta_3_top,0.0,11.349899,30.205097,0.821164,6446.134834,52.609805,0.807100,6335.737246
4,4,4,4940.337104,19145.252592,464.963852,412.443855,2135.646626,15.313616,16.445258,79.214303,...,-425.109802,eta_3_top,0.0,10.560859,43.978142,0.743621,5837.424167,33.727591,0.929415,7295.908523
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,995,995,3281.367780,15498.452667,577.306324,421.458974,4699.266956,29.736156,27.057742,93.172957,...,-4665.050781,eta_3_bot,0.0,6.351162,29.359562,0.816380,6408.583452,61.390866,1.226598,9628.790865
996,996,996,3666.643637,15455.753526,792.299652,471.784421,4931.536191,10.685627,14.709277,194.274833,...,-5082.087891,eta_3_bot,0.0,11.660351,44.900208,0.141711,1112.432057,71.919391,1.748475,13725.527284
997,997,997,6184.735919,11369.136905,645.636004,245.825059,3643.241327,28.115256,14.995675,101.134360,...,-4161.842285,eta_3_bot,0.0,8.950159,45.397966,0.711700,5586.843149,20.364398,0.845860,6639.997664
998,998,998,7491.926992,9743.524453,776.691431,387.895813,2073.204456,22.060343,17.066350,144.671295,...,-1316.269409,eta_3_bot,0.0,5.102009,56.696745,0.422417,3315.973081,15.671238,0.322936,2535.045656


In [111]:
### define environmental impact factors of my material

materials_concrete = {  'C30/37_average': {'Description': 'Durchschnittliche Zusammensetzungen für Beton in Deutschland (compression strength of 30 MPa) vom InformationsZentrum Beton GmbH',
                                         'EpdLink': 'https://www.oekobaudat.de/OEKOBAU.DAT/resource/sources/dc600cd9-9a55-414e-baf5-f36c54269803/Beton_der_Druckfestigkeitsklasse_C_3037_10616.pdf?version=00.02.000',
                                          'AP': 0.3431,
                                          'GWP': 218.71,
                                          'ADP-fossil': 1043.1 ,
                                          'ADP-minerals-metals': 0.000705978,
                                          'ODP': 0.000000059584079,
                                          'POCP': 0.016064, 
                                         },
                 
                        'C45/55_average': {'Description': 'durchschnittliche Zusammensetzungen für Beton in Deutschland (compression strength of 30 MPa) vom InformationsZentrum Beton GmbH',
                                         'EpdLink': 'https://www.oekobaudat.de/OEKOBAU.DAT/resource/sources/ef01ced5-dd04-43b8-80ce-5a5a7c421657/Beton_der_Druckfestigkeitsklasse_C_4555.pdf?version=00.01.000',
                                          'AP': 0.4321,
                                          'GWP': 285.71 ,
                                          'ADP-fossil': 1406.1 ,
                                          'ADP-minerals-metals': 0.001014978,
                                          'ODP': 0.000000077084079,
                                          'POCP': 0.025364, 
                                         },

                        'C50/60_average': {'Description': 'durchschnittliche Zusammensetzungen für Beton in Deutschland (compression strength of 30 MPa) vom InformationsZentrum Beton GmbH',
                                         'EpdLink': 'https://www.oekobaudat.de/OEKOBAU.DAT/resource/sources/892ca341-33e1-4136-81b9-ccd8418e9d73/Beton_der_Druckfestigkeitsklasse_C_5060_10526.pdf?version=00.03.000',
                                          'AP': 0.4481,
                                          'GWP': 299.71,
                                          'ADP-fossil': 1406.1 ,
                                          'ADP-minerals-metals': 0.001064978,
                                          'ODP': 0.000000083884079,
                                          'POCP': 0.028164, 
                                         }
                        }


materials_reinforcement= {  'Rebar Stahl Gerlafingen': {'Description': 'Rebar produced at Gerlafingen (Switzerland) B500B',
                                         'EpdLink': 'https://gruppobeltrame.com/wp-content/uploads/2021/02/EPD_Rebar-TOPAR-RC-rev1.pdf',
                                          'AP': 0.0009637,
                                          'GWP': 0.25054,
                                          'ADP-fossil': 9.8661,
                                          'ADP-minerals-metals': -0.0000015945439,
                                          'ODP': 0.000000037507,
                                          'POCP': 0.00078423, 
                                         }
                        }

In [112]:
# calc gwp
def calc_gwp(row):
    fcc=row['fcc']

    if fcc < 37.5:
        gwp_fac_conc = materials_concrete['C30/37_average']['GWP']
    elif 37.5 <= fcc < 47.5:
        gwp_fac_conc = materials_concrete['C45/55_average']['GWP']
    elif fcc >= 47.5:
        gwp_fac_conc = materials_concrete['C50/60_average']['GWP']
    else:
        raise ValueError(f"fcc value '{fcc}' does not fit the defined ranges.")
    
    gwp_fac_steel=materials_reinforcement['Rebar Stahl Gerlafingen']['GWP']

    gwp=gwp_fac_conc*(row['vol_plate_conc']+row['vol_walls_conc']) + gwp_fac_steel*(row['w_plate_steel']+row['w_walls_steel'])

    return gwp

In [113]:
# Calculate environemntal impact
def calc_envImpact(row, envImpact):
    fcc=row['fcc']

    if fcc < 37.5:
        eI_fac_conc = materials_concrete['C30/37_average'][envImpact]
    elif 37.5 <= fcc < 47.5:
        eI_fac_conc = materials_concrete['C45/55_average'][envImpact]
    elif fcc >= 47.5:
        eI_fac_conc = materials_concrete['C50/60_average'][envImpact]
    else:
        raise ValueError(f"fcc value '{fcc}' does not fit the defined ranges.")
    
    eI_fac_steel=materials_reinforcement['Rebar Stahl Gerlafingen'][envImpact]

    eI=eI_fac_conc*(row['vol_plate_conc']+row['vol_walls_conc']) + eI_fac_steel*(row['w_plate_steel']+row['w_walls_steel'])

    return eI

In [114]:
df_xyz['GWP']=df_xyz.apply(lambda row: calc_envImpact(row, envImpact='GWP'), axis=1)
df_xyz['GWP']

0      26502.735589
1      33129.329779
2      40117.512614
3      21314.817463
4      20285.446081
           ...     
995    23866.029758
996    37094.029816
997    22772.950988
998    23155.322480
999    15677.592038
Name: GWP, Length: 1000, dtype: float64

In [115]:
df_xyz['AP']=df_xyz.apply(lambda row: calc_envImpact(row, 'AP'), axis=1)
df_xyz['AP']

0      43.172495
1      52.490056
2      65.950366
3      40.731683
4      39.317430
         ...    
995    46.591689
996    64.777090
997    41.251122
998    38.066720
999    26.892383
Name: AP, Length: 1000, dtype: float64

In [116]:
df_xyz['ADP-minerals-metals']=df_xyz.apply(lambda row: calc_envImpact(row, 'ADP-minerals-metals'), axis=1)
df_xyz['ADP-minerals-metals']

0      0.079209
1      0.105244
2      0.117371
3      0.038084
4      0.033917
         ...   
995    0.038496
996    0.094910
997    0.050539
998    0.067741
999    0.041145
Name: ADP-minerals-metals, Length: 1000, dtype: float64

In [117]:
df_xyz['ADP-fossil']=df_xyz.apply(lambda row: calc_envImpact(row, 'ADP-fossil'), axis=1)
df_xyz['ADP-fossil']

0      176678.087777
1      199064.143643
2      276285.658703
3      212491.452481
4      210629.623955
           ...      
995    252888.109665
996    310652.828081
997    213099.694989
998    159483.357146
999    124485.514131
Name: ADP-fossil, Length: 1000, dtype: float64

In [118]:
df_xyz['ODP']=df_xyz.apply(lambda row: calc_envImpact(row, 'ODP'), axis=1)
df_xyz['ODP']

0      0.000233
1      0.000197
2      0.000391
3      0.000484
4      0.000497
         ...   
995    0.000607
996    0.000566
997    0.000464
998    0.000226
999    0.000224
Name: ODP, Length: 1000, dtype: float64

In [119]:
df_xyz['POCP']=df_xyz.apply(lambda row: calc_envImpact(row, 'POCP'), axis=1)
df_xyz['POCP']

0       7.071713
1       6.932669
2      11.478810
3      11.354266
4      11.547818
         ...    
995    14.034805
996    14.599385
997    11.440787
998     6.626716
999     5.931397
Name: POCP, Length: 1000, dtype: float64

In [120]:
df_xyz

,Unnamed: 0,ID,L,b1,t_p,t_w,h_w,d1_plate,d4_plate,s_plate,...,w_plate_steel,vol_walls_conc,vol_walls_steel,w_walls_steel,GWP,AP,ADP-minerals-metals,ADP-fossil,ODP,POCP
0,0,0,3264.911067,16765.132982,531.656436,621.775968,2604.161687,24.228625,13.427572,206.036846,...,1728.540762,54.292379,0.547000,4293.951771,26502.735589,43.172495,0.079209,176678.087777,0.000233,7.071713
1,1,1,5361.115418,17451.415474,744.304389,275.279703,3820.154356,17.838816,12.481430,218.545430,...,2011.240414,36.704199,0.383421,3009.852814,33129.329779,52.490056,0.105244,199064.143643,0.000197,6.932669
2,2,2,7197.928187,12610.611102,792.070735,496.244565,4273.476243,22.587214,26.028824,128.850732,...,6409.277052,53.486378,0.474510,3724.902312,40117.512614,65.950366,0.117371,276285.658703,0.000391,11.478810
3,3,3,4946.728918,16364.414176,373.131286,477.965439,3363.099660,24.607351,29.281128,135.563137,...,6446.134834,52.609805,0.807100,6335.737246,21314.817463,40.731683,0.038084,212491.452481,0.000484,11.354266
4,4,4,4940.337104,19145.252592,464.963852,412.443855,2135.646626,15.313616,16.445258,79.214303,...,5837.424167,33.727591,0.929415,7295.908523,20285.446081,39.317430,0.033917,210629.623955,0.000497,11.547818
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,995,995,3281.367780,15498.452667,577.306324,421.458974,4699.266956,29.736156,27.057742,93.172957,...,6408.583452,61.390866,1.226598,9628.790865,23866.029758,46.591689,0.038496,252888.109665,0.000607,14.034805
996,996,996,3666.643637,15455.753526,792.299652,471.784421,4931.536191,10.685627,14.709277,194.274833,...,1112.432057,71.919391,1.748475,13725.527284,37094.029816,64.777090,0.094910,310652.828081,0.000566,14.599385
997,997,997,6184.735919,11369.136905,645.636004,245.825059,3643.241327,28.115256,14.995675,101.134360,...,5586.843149,20.364398,0.845860,6639.997664,22772.950988,41.251122,0.050539,213099.694989,0.000464,11.440787
998,998,998,7491.926992,9743.524453,776.691431,387.895813,2073.204456,22.060343,17.066350,144.671295,...,3315.973081,15.671238,0.322936,2535.045656,23155.322480,38.066720,0.067741,159483.357146,0.000226,6.626716


In [121]:
xyz_file_name='CFBData\\{}_Batch\\{}_CFB_xyz.csv'.format(idx_s,idx_s)
df_xyz.to_csv(xyz_file_name)